# Merge 2021-12-03

This notebook merges the two datasets from 2022-12-03 into one H5 file. This is for convenient use in the analysis pipeline.

In [ ]:
import sys
import os
from os.path import join
import shutil
from datetime import datetime
import importlib
import json
import numpy as np
import pandas as pd
import h5py
import skimage
from tqdm import trange
from tqdm import tqdm
from matplotlib import pyplot as plt
from matplotlib import patches
import proplot as pplt

sys.path.append('../..')
from tools import energyVS06 as energy
from tools import image_processing as ip
from tools import plotting as mplt

In [ ]:
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['grid'] = False

In [ ]:
folder = '/Diagnostics/Data/Measurements/scan-xxpy-image-ypdE/2021-12-03/'
os.listdir(folder)

In [ ]:
t1 = '211203151919'
t2 = '211203192701'
timestamps = [t1, t2]

Merge the H5 files.

In [ ]:
files = []
for timestamp in timestamps:
    filename = join(folder, timestamp + '-scan-xxpy-image-ypdE.h5')
    files.append(h5py.File(filename))

In [ ]:
nrows = sum([len(file['scandata']) for file in files])

In [ ]:
dtype = files[0]['scandata'].dtype
dtype.names

Check iteration numbers.

In [ ]:
iteration1 = files[0]['scandata']['iteration'].copy()
iteration2 = files[1]['scandata']['iteration'].copy()

In [ ]:
ys = [file['scandata']['y_PositionSync'].copy() for file in files]

In [ ]:
fig, ax = pplt.subplots(figsize=(7, 2))
ax.plot(np.arange(len(ys[0])), ys[0])
ax.plot(np.arange(len(ys[1])) + len(ys[0]), ys[1])
window = 50
ax.format(xlim=(len(ys[0]) - window, len(ys[0]) + window))

In [ ]:
fig, ax = pplt.subplots(figsize=(7, 2))
ax.plot(np.arange(len(iteration1)), iteration1)
ax.plot(np.arange(len(iteration2)) + len(iteration1), iteration2)
window = 10000
ax.format(xlim=(len(iteration1) - window, len(iteration1) + window))

In [ ]:
iteration_offset = iteration1[-1]
iteration_offset

In [ ]:
i_offset = len(files[0]['scandata'])
i_offset

In [ ]:
file_out = h5py.File(join(folder, f'merged2-{t1}-{t2}-scan-xxpy-image-ypdE.h5'), mode='w')
data_out = file_out.create_dataset('scandata', (nrows,), dtype=dtype)
i_start = iteration_start = 0
for file in files:
    data = file['scandata']
    for i in trange(len(data)):
        for name in data.dtype.names:
            value = data[i, name]
            if name == 'iteration':
                value = value + iteration_start
            data_out[i + i_start, name] = value
    i_start += i_offset
    iteration_start += iteration_offset
    file.close()
file_out.close()

Copy the csv file for convenience. The csv file in the original data set contains all the planned points.

In [ ]:
shutil.copyfile(
    join(folder, f'{t1}-scan-xxpy-image-ypdE.csv'),
    join(folder, f'merged2-{t1}-{t2}-scan-xxpy-image-ypdE.csv'),
)

Copy the JSON file for convenience.

In [ ]:
shutil.copyfile(
    join(folder, f'{t1}-scan-xxpy-image-ypdE-metadata.json'),
    join(folder, f'merged2-{t1}-{t2}-scan-xxpy-image-ypdE-metadata.json'),
)